In [ ]:
!pip install haversine

In [ ]:
import pandas as pd
import folium
from geopy.distance import geodesic
import numpy as np
import pandas as pd
from haversine import haversine_vector, Unit

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
mclp_df = pd.read_csv("/content/drive/MyDrive/결과데이터/mclp_installed_bins.csv", encoding = 'CP949')
k_df = pd.read_csv("/content/drive/MyDrive/결과데이터/k-means_installed_bins.csv", encoding = 'CP949')
greedy_df = pd.read_csv("/content/drive/MyDrive/결과데이터/greedy_mclp_installed_bins.csv", encoding = 'CP949')
ensemble_df = pd.read_csv("/content/drive/MyDrive/결과데이터/ensemble_result.csv", encoding = 'utf-8')
all_df = pd.read_csv("/content/drive/MyDrive/결과데이터/all_df.csv",encoding = 'CP949')
random_df = all_df.sample(n=756, random_state=42)  # random_state는 재현성을 위해 고정

In [ ]:
# 전체 지점 가중치 기반 시각화

# 중심 위치 계산
center_lat = all_df['위도'].mean()
center_lon = all_df['경도'].mean()

# 지도 초기화
m = folium.Map(location=[center_lat, center_lon], zoom_start=14)

# 수요지점 시각화
for _, row in all_df.iterrows():
    weight = row['가중치']
    name = row['이름']
    lat = row['위도']
    lon = row['경도']

    folium.CircleMarker(
        location=[lat, lon],
        radius=4 + weight * 10,  # 가중치 기반 크기
        color='black',
        fill=True,
        fill_color='orange',
        fill_opacity=0.8,
        tooltip=name,
        popup=f"가중치: {round(weight, 3)}"
    ).add_to(m)

# 설치지점 시각화  df 이름 바꾸면서 확인가능(mclp_df, k_df, greedy_df, ensemble_df, random_df)
for _, row in k_df.iterrows():
    weight = row['가중치']
    name = row['이름']
    lat = row['위도']
    lon = row['경도']

    folium.Marker(
        location=[lat, lon],
        icon=folium.Icon(color='purple', icon='trash', prefix='fa'), ### 색깔바꾸기 가능(green yellow blue purple )
        tooltip=name,
        popup=f"설치 위치: {name}"
    ).add_to(m)


# 표시
m

Output hidden; open in https://colab.research.google.com to view.

커버율= 커버된 수요의 총 가중치/전체 수요의 총 가중치   ×100%

​


In [ ]:
# 수요지점 좌표 배열
demand_coords = all_df[['위도', '경도']].to_numpy()
demand_weights = all_df['가중치'].to_numpy()
total_weight = demand_weights.sum()

# 비교 대상 모델들
models = {
    'MCLP': mclp_df,
    'KMeans': k_df,
    'Greedy': greedy_df,
    'Ensemble': ensemble_df,
    'random' : random_df
}

# 결과 저장
coverage_results = []

# 각 모델 반복
for model_name, bin_df in models.items():
    bin_coords = bin_df[['위도', '경도']].to_numpy()

    # 거리 행렬 계산 (수요지점 수 x 설치지점 수)
    distances = haversine_vector(
        np.repeat(demand_coords, len(bin_coords), axis=0),
        np.tile(bin_coords, (len(demand_coords), 1)),
        Unit.METERS
    ).reshape(len(demand_coords), len(bin_coords))

    # 커버 여부 판별 (68.8m 이내에 하나라도 있으면 True)
    is_covered = (distances <= 68.8).any(axis=1)

    # 커버된 가중치 합산
    covered_weight = demand_weights[is_covered].sum()

    # 커버율 및 결과 저장
    coverage_ratio = covered_weight / total_weight
    coverage_results.append({
        '모델': model_name,
        '설치_개수': len(bin_df),
        '커버된_가중치': covered_weight,
        '전체_가중치': total_weight,
        '커버율(%)': round(coverage_ratio * 100, 2)
    })

# 데이터프레임으로 반환
coverage_df = pd.DataFrame(coverage_results)
print(coverage_df)

         모델  설치_개수     커버된_가중치      전체_가중치  커버율(%)
0      MCLP    580  229.448134  258.108295   88.90
1    KMeans    920  228.764775  258.108295   88.63
2    Greedy    800  243.061332  258.108295   94.17
3  Ensemble    756  238.779811  258.108295   92.51
4    random    756  162.887248  258.108295   63.11


In [ ]:
coverage_df

,모델,설치_개수,커버된_가중치,전체_가중치,커버율(%)
0,MCLP,580,229.448134,258.108295,88.90
1,KMeans,920,228.764775,258.108295,88.63
2,Greedy,800,243.061332,258.108295,94.17
3,Ensemble,756,238.779811,258.108295,92.51
4,random,756,162.887248,258.108295,63.11
